In [238]:
import pandas as pd

FILE = "INFLUD23-17-06-2024.csv"

INFLUD = pd.read_csv(FILE, sep=';')
POPULATION = pd.read_csv("StatePopulation.csv")
POPULATION.drop(columns=["STATE"], inplace=True)

C:\Users\kauan\AppData\Local\Temp\ipykernel_1820\2690819796.py:5: DtypeWarning: Columns (15,18,20,23,53,61,62,63,72,74,79,91,93,109,130,143,175,178) have mixed types. Specify dtype option on import or set low_memory=False.
  INFLUD = pd.read_csv(FILE, sep=';')


In [239]:
INFLUD["CLASSI_FIN"].groupby(INFLUD["CLASSI_FIN"]).size()

CLASSI_FIN
1.0     13432
2.0     48732
3.0      3633
4.0    153844
5.0     50246
Name: CLASSI_FIN, dtype: int64

In [240]:
df = INFLUD.copy()

# merge population data
df = df.merge(POPULATION, left_on='SG_UF_NOT', right_on='UF', how='left')

df = df[["DT_NOTIFIC", "SG_UF_NOT", "POPULATION", "CLASSI_FIN"]]
df["DT_NOTIFIC"] = pd.to_datetime(df["DT_NOTIFIC"], format='%d/%m/%Y').dt.dayofyear

df = df[(df["CLASSI_FIN"] != 3)]

df

,DT_NOTIFIC,SG_UF_NOT,POPULATION,CLASSI_FIN
0,11,PR,11597484,5.0
1,19,SC,7338473,5.0
2,23,SP,46289333,4.0
3,17,DF,3094325,2.0
4,19,PI,3322987,NaN
...,...,...,...,...
278977,340,SP,46289333,4.0
278978,363,GO,7113540,4.0
278979,349,BA,14930634,5.0
278980,355,PR,11597484,4.0


In [241]:
from sklearn.preprocessing import QuantileTransformer

ds = pd.DataFrame(df.groupby(["SG_UF_NOT", "DT_NOTIFIC"])["CLASSI_FIN"].size().reset_index(name='CNT_NOTIFIC'))

ds = ds.merge(POPULATION, left_on='SG_UF_NOT', right_on='UF', how="left")
ds.drop(columns=["SG_UF_NOT"], inplace=True)

risk_scaler = QuantileTransformer()
ds["RISK"] = ds["CNT_NOTIFIC"] / ds["POPULATION"] 
ds["RISK"] = risk_scaler.fit_transform(ds[["RISK"]])

ds.drop(columns=["CNT_NOTIFIC", "POPULATION"], inplace=True)

ds

,DT_NOTIFIC,UF,RISK
0,1,AC,0.427928
1,2,AC,0.934935
2,3,AC,0.195195
3,4,AC,0.709209
4,5,AC,0.885385
...,...,...,...
9331,360,TO,0.097097
9332,361,TO,0.476476
9333,362,TO,0.233734
9334,363,TO,0.363864
